### 사업보고서 "재무제표 주석" 문서 URL

In [1]:
import pandas as pd

# 사업보고서 목록
file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/dart_rpt/A_rpt_all.csv"
df_rpt_all = pd.read_csv(file_path, dtype=object)

* 대상 목록 확인

In [2]:
df_rpt_all['corp_cls'].value_counts()

corp_cls
K    9701
Y    5697
E    4592
N     693
Name: count, dtype: int64

In [3]:
# 회사코드 정렬
df_rpt_all = df_rpt_all.sort_values(by=['corp_code'], ascending=True)

In [15]:
# 회사코드 목록
# 전체
# list_all_codes = df_rpt_all['corp_code'].unique()     
# 유가증권(코스피) 기업
# list_all_codes = df_rpt_all[df_rpt_all['corp_cls'] == 'Y']['corp_code'].unique()
# 유가증권 기업 + 결산년도 2017~2021
list_all_codes = df_rpt_all.loc[(df_rpt_all.corp_cls == 'Y') & (df_rpt_all.report_nm.str[-8:-4] >= '2017') & (df_rpt_all.report_nm.str[-8:-4] <= '2021'), ]['corp_code'].unique()
len(list_all_codes)

812

* 이미 재무제표 주석 문서 URL을 찾은 것은 대상에서 제외

In [16]:
# 이미 재무제표 주석 문서 URL을 찾은 회사코드 목록
file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/fs_notes/fs_notes_all.csv"
fs_notes_all = pd.read_csv(file_path, dtype=object)
list_own_codes = fs_notes_all['corp_code'].unique()
len(list_own_codes)

24

In [17]:
# 리스트에서 중복 제거 (이미 corp info가 있는 경우 대상에서 제외)
list_target_codes = list(set(list_all_codes) - set(list_own_codes))
len(list_target_codes)

792

* 리스트 분할

In [18]:
list_cnt = len(list_target_codes)
n = 100
list_of_lists = [list_target_codes[i * n:(i + 1) * n] for i in range((list_cnt + n - 1) // n )] 
len(list_of_lists)

8

* 재무제표 주석 문서 가져오기

In [19]:
import traceback
from time import sleep
import OpenDartReader
api_key = 'f2e08d4ed3de0ba3d5cbf59c04c223e02b1751a2'
dart = OpenDartReader(api_key)

In [20]:
# 파일 저장 위치
path_dir = "D:/PythonProject/data-gatherer/dart_fs_notes/fs_notes/"

for corp_codes in list_of_lists:
    for corp_code in corp_codes:
        df_biz_rpt = df_rpt_all.loc[df_rpt_all.corp_code == corp_code,].sort_values(by=['rcept_dt'], ascending=True)
        subreports = []
        for i, row in df_biz_rpt.iterrows():
            sleep(0.1)
            try:
                rpt_name = row['report_nm']
                rpt_num = row['rcept_no']        
                rpt_url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + rpt_num
                df_doc = dart.sub_docs(rpt_num)
                df_notes = df_doc.loc[(df_doc.title.str.contains('주석')) & (~df_doc.title.str.contains('연결')),]
                if df_notes.size == 0:
                    subreports.append({'corp_code' : row['corp_code'], 'stock_code' : row['stock_code'], 'corp_cls' : row['corp_cls'], 'corp_name' : row['corp_name'],
                                       'rpt_num' : rpt_num, 'rpt_name' : rpt_name, 'rpt_url' : rpt_url,
                                       'doc_title' : None, 'doc_url' : None, 'doc_result' : 'No Financial Statement Notes'})                 
                else:
                    rpt_title = df_notes.iloc[0]['title']
                    rpt_notes_url = df_notes.iloc[0]['url']
                    subreports.append({'corp_code' : row['corp_code'], 'stock_code' : row['stock_code'], 'corp_cls' : row['corp_cls'], 'corp_name' : row['corp_name'],
                                       'rpt_num' : rpt_num, 'rpt_name' : rpt_name, 'rpt_url' : rpt_url, 
                                       'doc_title' : rpt_title, 'doc_url' : rpt_notes_url, 'doc_result' : 'OK'}) 
            except Exception as e:
                print(row['rcept_no'])
                print(row['report_nm'])
                print('http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + row['rcept_no'])
                err_msg = traceback.format_exc()
                print(err_msg)
                break
        
        if len(subreports) == 0:
            break
        else:
            df_subreports = pd.DataFrame(subreports)
            file_name = corp_code + '.csv'
            df_subreports.to_csv(path_dir + file_name, index=False)

20210318001008
사업보고서 (2020.12)
http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20210318001008
Traceback (most recent call last):
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\urllib3\connectionpool.py", line 790, in urlopen
    response = self._make_request(
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\urllib3\connection.py", line 454, in getresponse
    httplib_response = super().getresponse()
  File "C:\Users\kyungho\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 1374, in getresponse
    response.begin()
  File "C:\Users\kyungho\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "C:\Users\kyungho\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 2